# Feature extraction in COGNITION
some critical features

200 Hz

broadband [0,40]

In [1]:
#external libraries
import os
import dotenv
import pandas as pd
import numpy as np
from scipy import signal as sg
import pickle as pkl
import time
import matplotlib.pyplot as plt

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']

In [2]:
#define features path
cognipath=datapath+"/COGNITION"
featurespath=cognipath+"/features"
        
windowDurations=[0.5,1,1.5,2]

## Load data

In [3]:
signals, annotations, signalsMetadata = loadCOGNITIONSpindles(cognipath, returnSignals=True)

In [4]:
signalsMetadata

,filename,channel,subjectId,duration,samplerate,isOriginalSamplerate,database
0,COG001_canal_C3.txt,C3-M2,0001,39600,200,False,COGNITION
1,COG002_canal_C3.txt,C3-M2,0002,39600,200,False,COGNITION
2,COG003_canal_C3.txt,C3-M2,0003,39600,200,False,COGNITION
3,COG004_canal_C3.txt,C3-M2,0004,39600,200,False,COGNITION
4,COG005_canal_C3.txt,C3-M2,0005,39600,200,False,COGNITION
5,COG006_canal_C3.txt,C3-M2,0006,39600,200,False,COGNITION
6,COG007_canal_C3.txt,C3-M2,0007,39600,200,False,COGNITION
7,COG008_canal_C3.txt,C3-M2,0008,39600,200,False,COGNITION
8,COG009_canal_C3.txt,C3-M2,0009,39600,200,False,COGNITION


## Preprocess

In [5]:
samplerate=200

broadband in 0-40Hz, previous experiments (up tp 50Hz, are now in folders as .../features_old/...)

In [6]:
def preprocessVector(vector,samplerate):
    
    #1. Lowpass 40Hz
    vector=filterBand(vector,[0,40],samplerate,filterOrder=4)
    #2. Robust Z-score
    from sklearn.preprocessing import RobustScaler
    vector_preprocessed=RobustScaler().fit_transform(vector.reshape(-1, 1) )
    return vector_preprocessed. flatten()

In [7]:
for subject,signal in signals.items():
    signals[subject]=preprocessVector(signal,samplerate)

## Extract features

### folder estructure

In [8]:
fspath=featurespath+'/'+str(samplerate)+'fs/'
if not os.path.isdir(fspath):
    os.mkdir(fspath)
for window in windowDurations:
    windowPath=fspath+str(window)+'win'
    if not os.path.isdir(windowPath):
        os.mkdir(windowPath)
    for subject in signalsMetadata.subjectId:
        subjectPath=windowPath+'/'+subject
        if not os.path.isdir(subjectPath):
            os.mkdir(subjectPath)

### band definition

In [9]:
bands={
    'delta1':[0.1,2],
    'delta2':[2,4],
    'theta':[4,8],
    'alpha':[8,13],
    'sigma':[11,16],
    'beta1':[13,19],
    'beta2':[19,30]
    }

### computation

In [10]:
for window in windowDurations:
    windowPath=featurespath+'/'+str(samplerate)+'fs/'+str(window)
    for ind, row in signalsMetadata.iterrows():
        subject=row.subjectId
        subjectPath=windowPath+'/'+subject
        signal=signals[subject]
        #need to define the time vector for each signal:
        timepoints=np.arange(len(signal))/samplerate
        
        #6. Hjort Activity
        characteristic='hjortActivity'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortActivity(signal,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortActivity(filtered,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #7. Hjort Mobility
        characteristic='hjortMobility'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortMobility(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortMobility(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #8. Hjort Complexity
        characteristic='hjortComplexity'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortComplexity(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortComplexity(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #9. PetrosianFractalDimension
        characteristic='petrosian'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=petrosianFractalDimension(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=petrosianFractalDimension(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        
        #12. sigma index
        characteristic='sigmaIndex'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=sigmaindex(signal,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
            

saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortActivity_broadband.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortActivity_delta1.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortActivity_delta2.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortActivity_theta.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortActivity_alpha.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortActivity_sigma.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortActivity_beta1.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortActivity_beta2.fd


C:\Users\GIB\Documents\GitHub\SpinCo\src\spinco.py:333: RuntimeWarning: divide by zero encountered in divide
  mobility=np.sqrt(moveVarCentered(vectorGradient,windowSampleCount)/moveVarCentered(vector,windowSampleCount))
C:\Users\GIB\Documents\GitHub\SpinCo\src\spinco.py:333: RuntimeWarning: invalid value encountered in divide
  mobility=np.sqrt(moveVarCentered(vectorGradient,windowSampleCount)/moveVarCentered(vector,windowSampleCount))


saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortMobility_broadband.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortMobility_delta1.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortMobility_delta2.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortMobility_theta.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortMobility_alpha.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortMobility_sigma.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortMobility_beta1.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortMobility_beta2.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortComplexity_broadband.fd
saving feature: D:/SpinCo/COGNITION/features/200fs/0.5win/0001/0.5_0001_hjortComplexity_delta1.fd
saving feature: D:/SpinCo/COGNITION